In [180]:
import numpy as np

class Convolution:
    #include init? idk
        
    # Given set of filters (features) #odd x #odd, iterate over input and multiply
    # img_in: a square NxM #even x #even matrix representing one of the RGB values of the input
    # features: filters/features that we compare the input against
    # step_size: fixed to 1 (does not have any effect)
    # returns a NxM matrix where each value is the result of multiplying
    # filter with the input, then dividing by the size of the filter
    def forward_prop(self, img_in, features):
        N = len(img_in)
        M = len(img_in[0])
        # if step size doesn't match up -> padding()
        
        # gives us amount of padding needed
        pad_N = (len(features) - 1)
        pad_M = (len(features[0]) - 1)
        
        # padded img
        img_in = self.padding(img_in, N + pad_N, M + pad_M)
        # print(img_in)
        
        # NxM output array (square)
        out = np.zeros((N, M))
        
        # iterate thru the input and output to corresponding part in out
        for i in range(0, N):
            for j in range (0, M):
                # Take dot product of the flattened filter and part of input it is covering
                out[i, j] = np.dot(np.ndarray.flatten(img_in[i:i+len(features),
                            j:j+len(features[0])]), np.ndarray.flatten(features))
        
        return out
    
    # loss = loss gradient from the next layer (feeding back to previous layer)
    # features = previous/original kernel/filter
    # learning_rate = alpha or how much change we want (the same across the network)
    # loss_size = img_size
    def back_prop(self, img_in, loss, features, learning_rate):
        dL_dF = np.zeros(np.shape(features))
        X = img_in
        N = len(X)
        M = len(X[0])
        
        pad_N = (len(features) - 1)
        pad_M = (len(features[0]) - 1)

        # padded img
        X = self.padding(X, N + pad_N, M + pad_M)
        print("padded img", X)
        
        # update filter
        # X is the image @ this stage
        # chain rule: dL_dF = dL_dO * dO_dF
        # but dL_dO is the input loss
        # and dO_dF results in the image X
        # dL_dF = convolution 2d between X (image @ this stage) and loss   
        # ex. dL_dF 3x3 =  Conv image 4x4 , loss 3x3
        for i in range(0, N):
            for j in range (0, M):
                # print(np.ndarray.flatten(X[i:i+len(loss), j:j+len(loss[0])]))
                dL_dF[i, j] = np.dot(np.ndarray.flatten(X[i:i+len(loss),
                            j:j+len(loss[0])]), np.ndarray.flatten(loss))
        print("dL_dF")
        print(dL_dF)
        
        
        # update the image X
        # need to rotate the filter 180deg and pad it
        # convolution from right to left and bottom to top
        # F33 F32 F31
        # F23 F22 F21
        # F13 F12 F11
        dL_dX = np.zeros((N, M))
        # rotate the filter 180deg
        rot_features = np.rot90(features, 2)
        # pad the filter 
        loss = self.padding(loss, N + pad_N, M + pad_M)
        print("padded loss")
        print(loss)
        # dL_dX = convolution 2d between rot180(Filter) and loss 
        # convolution from right to left and bottom to top
        for i in range(N - 1, -1, -1):
            for j in range (M - 1, -1, -1):
                #print(i,j)
                #print(np.ndarray.flatten(loss[i:i+len(rot_features), j:j+len(rot_features[0])]))
                dL_dX[i, j] = np.dot(np.ndarray.flatten(loss[i:i+len(rot_features),
                            j:j+len(rot_features[0])]), np.ndarray.flatten(rot_features))
        
        print("dL_dX")
        print(dL_dX)
        # F = F - a * dL/dF
        features = features - learning_rate * dL_dF
        print("updated_features")
        print(features)
        return features, X
    
    # Add 0s to the border of orig_img as needed to achieve NxM matrix
    # assume orig_img is a square matrix with #odd x #odd
    def padding(self, orig_img, N, M):
        starting_row = (N - orig_img.shape[0]) // 2
        starting_column = (M - orig_img.shape[1]) // 2
        pad_arr = np.zeros((N, M))
        pad_arr[starting_row:starting_row+orig_img.shape[0], starting_column:starting_column+orig_img.shape[1]] = orig_img
        return pad_arr
    
# Can implement either max or avg pooling, going w max for now
class Pooling: 
    # Go thru sections of the original matrix and only take the highest value
    # Put this into a new PxQ matrix
    # Pool dim is a single int representing both size of the 'pool filter' and
    # the stride. Ex: 2 -> 2x2 pooling with stride 2
    def forward_prop(self, img_in, pool_dim):
        # if step size doesn't match up -> padding()  
        # gives us amount of padding needed
        pad_N = (len(img_in) - pool_dim) % pool_dim
        pad_M = (len(img_in[0]) - pool_dim) % pool_dim
        
        if (pad_N is not 0) or (pad_M is not 0):
            img_in = self.padding(img_in, len(img_in)+pad_N, len(img_in[0])+pad_M)
        
        # reduce by a factor of whatever stride is
        P = int(len(img_in) / pool_dim)
        Q = int(len(img_in[0]) / pool_dim)
        out = np.zeros((P, Q))
        # loop thru input matrix
        for i in range(0, P):
            for j in range(0, Q):
                # get largest value in pool
                vert = i * pool_dim
                horiz = j * pool_dim
                pool = img_in[vert:vert+pool_dim, horiz:horiz+pool_dim]
                #print(pool)
                out[i,j] = np.amax(pool)
    
        return out
    
    def back_prop(self):
        
        return None
    def padding(self, orig_img, N, M):
        starting_row = int((N - orig_img.shape[0]) // 2)
        starting_column = int((M - orig_img.shape[1]) // 2)
        pad_arr = np.zeros((N, M))
        pad_arr[starting_row:starting_row+orig_img.shape[0], starting_column:starting_column+orig_img.shape[1]] = orig_img
        return pad_arr
    
class ReLU():
    def forward_prop(self, img_in):
        for i in range(0, len(img_in)):
            for j in range(0, len(img_in[0])):
                if img_in[i, j] < 0:
                    img_in[i, j] = 0
        return img_in
    def back_prop():
        return None
    
class Sigmoid_Act():
    def forward_prop():
        return None
    def back_prop():
        return None
          

In [181]:
# tests
def test_conv_forward():
    test_arr = np.array(([5, 5, 2, 7],
                         [5, 5, 5, 0],
                         [5, 5, 5, 22],
                         [1, 2, 3, 4]))
    feat_arr = np.array(([2, -1, 0], 
                         [2, 1, 1],
                         [1, 0, -1]))
    conv_layer = Convolution()
    print(conv_layer.forward_prop(test_arr, feat_arr))
    
def test_pool_forward():
    test_arr = np.array(([5, 5, 2],
                         [5, 6, 2],
                         [5, 5, 4]))
    pool_dim = 2
    pool_layer = Pooling()
    print(pool_layer.forward_prop(test_arr, pool_dim))
    
def test_ReLU():
    test_arr = np.array(([5, -1, -1],
                         [-4, 6, 2],
                         [5, 0, -3333]))
    ReLU_layer = ReLU()
    print(ReLU_layer.forward_prop(test_arr))
    
    
test_conv_forward()
test_pool_forward()
test_ReLU()

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  5.  5.  2.  7.  0.]
 [ 0.  5.  5.  5.  0.  0.]
 [ 0.  5.  5.  5. 22.  0.]
 [ 0.  1.  2.  3.  4.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 5. 17. 24. 16.]
 [ 0. 25.  6. 12.]
 [ 3. 23. 40. 45.]
 [-2. 12. 16. -2.]]
[[6. 2.]
 [5. 4.]]
[[5 0 0]
 [0 6 2]
 [5 0 0]]


In [86]:
# tests 2
# def padding(self, orig_img, N, M)
def test_padding():
    conv_layer = Convolution()
    test_arr = np.array(([5, 5, 2, 7],
                         [5, 5, 5, 0],
                         [5, 5, 5, 22],
                         [1, 2, 3, 4]))
    print(conv_layer.padding(test_arr,6,6))
    test_arr = np.array(([5, 5, 2, 7],
                         [5, 5, 5, 0],
                         [5, 5, 5, 22],
                         [1, 2, 3, 4]))
    print(conv_layer.padding(test_arr,8,8))
    test_arr = np.array(([2, 3],
                         [4, 5]))
    print(conv_layer.padding(test_arr,4,4))
    
print("TEST: padding")
test_padding()

TEST: padding
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  5.  5.  2.  7.  0.]
 [ 0.  5.  5.  5.  0.  0.]
 [ 0.  5.  5.  5. 22.  0.]
 [ 0.  1.  2.  3.  4.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  5.  2.  7.  0.  0.]
 [ 0.  0.  5.  5.  5.  0.  0.  0.]
 [ 0.  0.  5.  5.  5. 22.  0.  0.]
 [ 0.  0.  1.  2.  3.  4.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[0. 0. 0. 0.]
 [0. 2. 3. 0.]
 [0. 4. 5. 0.]
 [0. 0. 0. 0.]]


In [182]:
# tests 3
# def back_prop(self, img_in, loss, features, learning_rate)
def test_conv_backward():
    test_arr = np.array(([5, 5, 2, 7],
                         [5, 5, 5, 0],
                         [5, 5, 5, 22],
                         [4, 3, 2, 1]))
    feat_arr = np.array(([2, -1, 0], 
                         [2, 1, 1],
                         [1, 0, -1]))
    loss = np.array(([5, 3, 1, 9], 
                     [2, 4, 6, -1],
                     [9, 7, 5, -3],
                     [-1,-2,-3,-4]))
    conv_layer = Convolution()
    conv_layer.back_prop(test_arr, loss, feat_arr, 0.01)
    
print("TEST: back_prop")
test_conv_backward()

TEST: back_prop
padded img [[ 0.  0.  0.  0.  0.  0.]
 [ 0.  5.  5.  2.  7.  0.]
 [ 0.  5.  5.  5.  0.  0.]
 [ 0.  5.  5.  5. 22.  0.]
 [ 0.  4.  3.  2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.]]


ValueError: shapes (12,) and (16,) not aligned: 12 (dim 0) != 16 (dim 0)